In [31]:
import argparse
import os
import numpy as np
import math
import sys

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [32]:
print(os.getcwd())
currentPath = os.getcwd().split('/')

indexOf = currentPath.index('CDCGAN')
rootPath = '/'.join(currentPath[:indexOf+1])
os.chdir(rootPath)
root = os.getcwd()

print(os.getcwd())

/workspace/CDCGAN
/workspace/CDCGAN


In [33]:
os.makedirs("images", exist_ok=True)

parser = argparse.ArgumentParser()
parser.add_argument("--n_epochs", type=int, default=1000, help="number of epochs of training")
parser.add_argument("--batch_size", type=int, default=128, help="size of the batches")
parser.add_argument("--lr", type=float, default=0.00005, help="learning rate")
parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
parser.add_argument("--latent_dim", type=int, default=100, help="dimensionality of the latent space")
parser.add_argument("--img_size", type=int, default=28, help="size of each image dimension")
parser.add_argument("--channels", type=int, default=1, help="number of image channels")
parser.add_argument("--n_critic", type=int, default=1, help="number of training steps for discriminator per iter")
parser.add_argument("--clip_value", type=float, default=0.01, help="lower and upper clip value for disc. weights")
parser.add_argument("--sample_interval", type=int, default=100, help="interval betwen image samples")
# opt = parser.parse_args()
opt, unknown = parser.parse_known_args()
print(opt)


img_shape = (opt.channels, opt.img_size, opt.img_size)

cuda = True if torch.cuda.is_available() else False

Namespace(n_epochs=1000, batch_size=128, lr=5e-05, n_cpu=8, latent_dim=100, img_size=28, channels=1, n_critic=5, clip_value=0.01, sample_interval=100)


In [34]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(opt.latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.shape[0], *img_shape)
        return img


In [35]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
        )

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        validity = self.model(img_flat)
        return validity

In [36]:
# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()

In [37]:
# # Configure data loader
# os.makedirs("../../data/mnist", exist_ok=True)
# dataloader = torch.utils.data.DataLoader(
#     datasets.MNIST(
#         "../../data/mnist",
#         train=True,
#         download=True,
#         transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]),
#     ),
#     batch_size=,
#     shuffle=True,
# )

In [38]:
from packages.dataHandlers.datasetMannager import datasetMannager
from torchvision import transforms

transform = transforms.Compose([
        transforms.Resize(opt.img_size),
        transforms.Grayscale(num_output_channels=opt.channels),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ])

dataMannager = datasetMannager(transform, opt.batch_size, 'LETTERS')
data_module = dataMannager.getDataModule()
dataloader = data_module.getTrainDataLoader()

In [39]:
# Optimizers
optimizer_G = torch.optim.RMSprop(generator.parameters(), lr=opt.lr)
optimizer_D = torch.optim.RMSprop(discriminator.parameters(), lr=opt.lr)

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [40]:
# ----------
#  Training
# ----------

batches_done = 0
for epoch in range(opt.n_epochs):

    for i, (imgs, _) in enumerate(dataloader):

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))))

        # Generate a batch of images
        fake_imgs = generator(z).detach()
        # Adversarial loss
        loss_D = -torch.mean(discriminator(real_imgs)) + torch.mean(discriminator(fake_imgs))

        loss_D.backward()
        optimizer_D.step()

        # Clip weights of discriminator
        for p in discriminator.parameters():
            p.data.clamp_(-opt.clip_value, opt.clip_value)

        # Train the generator every n_critic iterations
        if i % opt.n_critic == 0:

            # -----------------
            #  Train Generator
            # -----------------

            optimizer_G.zero_grad()

            # Generate a batch of images
            gen_imgs = generator(z)
            # Adversarial loss
            loss_G = -torch.mean(discriminator(gen_imgs))

            loss_G.backward()
            optimizer_G.step()

            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                % (epoch, opt.n_epochs, batches_done % len(dataloader), len(dataloader), loss_D.item(), loss_G.item())
            )

        if batches_done % opt.sample_interval == 0:
            path =  f"{root}/results/test_Architecture/"
            if not os.path.exists(path): os.mkdir(path)

            path = f"{path}test_WGAN/"
            if not os.path.exists(path): os.mkdir(path)

            save_image(gen_imgs.data[:25], path+f"epoch_{batches_done}.jpg", nrow=5, normalize=True)
        batches_done += 1

[Epoch 0/1000] [Batch 0/10] [D loss: -0.057335] [G loss: -0.010188]
[Epoch 0/1000] [Batch 5/10] [D loss: -0.147198] [G loss: -0.011410]
[Epoch 1/1000] [Batch 0/10] [D loss: -0.488288] [G loss: -0.015866]
[Epoch 1/1000] [Batch 5/10] [D loss: -1.019452] [G loss: -0.031409]
[Epoch 2/1000] [Batch 0/10] [D loss: -1.678798] [G loss: -0.061980]
[Epoch 2/1000] [Batch 5/10] [D loss: -2.417656] [G loss: -0.109423]
[Epoch 3/1000] [Batch 0/10] [D loss: -3.185242] [G loss: -0.179103]
[Epoch 3/1000] [Batch 5/10] [D loss: -3.998581] [G loss: -0.270837]
[Epoch 4/1000] [Batch 0/10] [D loss: -4.804173] [G loss: -0.383402]
[Epoch 4/1000] [Batch 5/10] [D loss: -5.594620] [G loss: -0.515907]
[Epoch 5/1000] [Batch 0/10] [D loss: -6.307241] [G loss: -0.700271]
[Epoch 5/1000] [Batch 5/10] [D loss: -7.077980] [G loss: -0.906973]
[Epoch 6/1000] [Batch 0/10] [D loss: -7.765040] [G loss: -1.122947]
[Epoch 6/1000] [Batch 5/10] [D loss: -8.485178] [G loss: -1.367044]
[Epoch 7/1000] [Batch 0/10] [D loss: -9.106074] 